In [1]:
import os
import importlib 
import pprint

import numpy as np
import pandas as pd
import xarray as xr

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad


from IPython import display

# 2 x $CH_4$

In [2]:
pd.set_option('max_colwidth', -1)

In [7]:
def params_clirad_nonoverlap():
    '''
    Define clirad-lw input parameter dictionaries
    for 2xch4, ch4-only runs.
    '''
    commitnumber = '523cbb7'
    molecule = {'ch4': 2 * 1.8e-6}
    bands = [9] # these are the only bands with ch4
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for band in bands:
        for atmpro, tsfc in atmpros:
            param = {'commitnumber': commitnumber,
                     'molecule': molecule,
                     'band': [band],
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            params.append(param)
    return params



def params_clirad_overlap():
    '''
    Define clirad-lw input parameter dictionaries
    for 2xn2o, n2o overlapping with others.
    '''
    commitnumber = '523cbb7'
    
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    molecule_bands = [
        ({'h2o': 'atmpro', 'co2': 400e-6, 'n2o': 2 * 3.2e-7}, [3]),
        ({'h2o': 'atmpro', 'n2o': 2 * 3.2e-07, 'ch4': 1.8e-06}, [9]),
        ({'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 2 * 1.8e-06}, [9])    
    ]
    
    params = []
    for molecule, bands in molecule_bands:
        for band in bands:
            for atmpro, tsfc in atmpros:
                param = {'commitnumber': commitnumber,
                         'molecule': molecule,
                         'band': [band],
                         'atmpro': atmpro,
                         'tsfc': tsfc}
                params.append(param)
    return params





def all_mls_params():
    '''
    Gather all non-overlap and overlap params that are
    for atmosphere profile mls.  
    '''    
    params_nonoverlap = [p for p in params_clirad_nonoverlap()
                         if p['atmpro'] == 'mls']
    params_overlap = [p for p in params_clirad_overlap()
                      if p['atmpro'] == 'mls']
    params = params_nonoverlap + params_overlap
    return params

In [8]:
def get_hrefs_clirad(param=None):
    atmpros = ('mls', 'saw', 'trp')
        
    hrefs = []
    for atmpro in atmpros:
        p = param.copy()
        p['atmpro'] = atmpro
        url = pipeline.nbviewer_url(param=p, setup=setup)
        
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs



def tb_hrefs_clirad():
    df = pd.DataFrame()
    
    for param in all_mls_params():
        
        hrefs = get_hrefs_clirad(param=param)
        
        bands = [str(b) for b in param['band']]
        
        df.loc['+'.join(param['molecule']), 
               '+'.join(bands)] = hrefs
        
    df = df.fillna('')
    
    bmap = utils_clirad.mapband_new2old()
    lblnew_bands = [bmap[int(b)] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([lblnew_bands, df.columns],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    

    return df
        


In [9]:
def script():
    df = tb_hrefs_clirad()
    
    s_html = df.to_html(escape=False)
    
    display.display(display.HTML(s_html))

In [10]:
script()

lblnew,7,3a
clirad,9,3
ch4,mls saw trp,
h2o+co2+n2o,,mls saw trp
h2o+n2o+ch4,mls saw trp,


In [11]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')